# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
query = establishments.find({"scores.Hygiene": 20})

# Use count_documents to display the number of documents in the result
establishments.count_documents({"scores.Hygiene": 20})

# Display the first document in the results using pprint
pprint(query[0])

{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6667a654387ccbc227db2d6b'),
 'geocode': {'latitude': 50.769705, 'longitude': 0.27694},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate

In [10]:
# Convert the result to a Pandas DataFrame
df_hygiene = pd.DataFrame(list(query))

# Display the number of rows in the DataFrame
print(len(df_hygiene))

# Display the first 10 rows of the DataFrame
df_hygiene.head(10)


0


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [11]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = establishments.find(
    {"LocalAuthorityName": {"$regex": "London"},
     "RatingValue": {"$gte": 4}}
)

# Use count_documents to display the number of documents in the result
establishments.count_documents(
    {"LocalAuthorityName": {"$regex": "London"},
     "RatingValue": {"$gte": 4}}
)

# Display the first document in the results using pprint
pprint(query[0])

{'AddressLine1': 'Oak Apple Farm Building 103 Sheernes Docks',
 'AddressLine2': 'Sheppy Kent',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': "Charlie's",
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4627.439467780196,
 'FHRSID': 621707,
 'LocalAuthorityBusinessID': 'PI/000025307',
 'LocalAuthorityCode': '508',
 'LocalAuthorityEmailAddress': 'publicprotection@cityoflondon.gov.uk',
 'LocalAuthorityName': 'City of London Corporation',
 'LocalAuthorityWebSite': 'http://www.cityoflondon.gov.uk/Corporation/homepage.htm',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'ME12',
 'RatingDate': '2021-10-18T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6667a654387ccbc227db4788'),
 'geocode': {'latitude': 51.369321, 'longitude': 0.508551},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/621707',
            'rel': 'self'}]

In [13]:
# Convert the result to a Pandas DataFrame
df_london_high_rating = pd.DataFrame(list(query))

# Display the number of rows in the DataFrame
print(len(df_london_high_rating))

# Display the first 10 rows of the DataFrame
df_london_high_rating.head(10)

0


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [15]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
# Retrieve "Penang Flavours" restaurant
penang_flavours = establishments.find_one({"BusinessName": "Penang Flavours"})

latitude = penang_flavours["geocode"]["latitude"]
longitude = penang_flavours["geocode"]["longitude"]

query = establishments.find(
    {
        "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
        "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
        "RatingValue": 5
    }
).sort("scores.Hygiene", 1).limit(5)

# Print the results
for est in query:
    pprint(est)


{'AddressLine1': 'Restaurant And Premises 107A Plumstead High Street',
 'AddressLine2': '',
 'AddressLine3': 'Plumstead',
 'AddressLine4': 'Greenwich',
 'BusinessName': 'Howe and Co Fish and Chips - Van 17',
 'BusinessType': 'Mobile caterer',
 'BusinessTypeID': 7846,
 'ChangesByServerID': 0,
 'Distance': 4646.95593107927,
 'FHRSID': 1380578,
 'LocalAuthorityBusinessID': '14425',
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'SE18 1SE',
 'RatingDate': '2021-11-11T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6667a655387ccbc227db85b0'),
 'geocode': {'latitude': 51.4875335693359, 'longitude': 0.0925370007753372},
 'links': [{'href': 'http://api.ratings.food.gov.uk/establishments/1380578',
            'rel': 'self'}],


In [17]:
# Convert result to Pandas DataFrame
query_results = [est for est in query]
df_nearby = pd.DataFrame(query_results)
df_nearby.head()

""


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [18]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 10}
]
results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
pprint(len(results))
# Print the first 10 results
pprint(results)


10
[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [19]:
# Convert the result to a Pandas DataFrame
df_hygiene_0 = pd.DataFrame(results)
# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame
df_hygiene_0.head(10)

,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
